# data processing
refactored to df

## imports

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
import sys
import os
from pathlib import Path
import configparser
config = configparser.ConfigParser()
config.read_file(open('privateconfig'))
resdir = Path(config['Datafolder']['data'])
workdir = Path(config['Codefolder']['workspace'])
os.chdir(workdir)

In [4]:
# analysis
from scipy.io import loadmat
from sklearn.decomposition import FastICA
from sklearn.datasets import make_regression
from sklearn.model_selection import KFold
from sklearn.linear_model import LassoCV, Lasso
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr


In [5]:
# misc
import pickle
from collections import defaultdict

In [6]:
# task
from env_config import Config
from firefly_task import ffacc_real
# from monkey_functions import *
# from InverseFuncs import *
from stable_baselines3 import TD3
import torch


In [7]:
from neural_plot_ult import *
import time
tic=time.time()
import warnings
warnings.filterwarnings('ignore')

# Pre IRC

convert the mat data file (with neural data) into (states, actions, tasks) for IRC.


## prepare

In [8]:
# const
bin_size = 17  # how many bin of DT. about 0.1 s
# num_bins = 24  # how many bins to use. use 2.4 s and discard the long trials.
# monkey_height = 10
DT = 0.006  # DT for raw data
# reward_boundary = 65
areas = ['PPC', 'PFC', 'MST']
worldscale = 200


m = 'm51'
folder = 'm51_mat_ruiyi'
dens = [0.0001, 0.0005, 0.001,  0.005]

locals().update({m: {}})
figure_path = resdir/'figures'
# datapaths = [i for i in Pa1th(resdir/'mat_ruiyi').glob(f'{m}*.mat')]
datapaths = [i for i in Path(resdir/folder).glob(f'{m}*.mat')]
session=[int(a.stem[-2:]) for a in datapaths]


## from raw data file: task relavent variables and neural

In [9]:
# load raw data
df=pd.DataFrame() 
for idx, datapath in enumerate(datapaths):
    if datapath.stem[-1].isalpha():
        continue
    data = loadmat(datapath)
    eval(m)[datapath.stem] = data

    # df
    sessdf=pd.DataFrame() 
    sessdf['trial']=np.arange((len(data['trials_behv'][0])))
    sessdf['session']=int(datapath.stem[(datapath.stem).find('s')+1:])
    df=pd.concat([df, sessdf])

In [10]:
sessdata = defaultdict(list)

for key, data in eval(m).items():
    sess = int(key[-2:])
    if key[-1].isalpha():
        continue

    trials_behv = data['trials_behv'][0]
    trials_units = data['units'][0]
    units_area = np.array([v[0] for v in trials_units['brain_area']])
    trials_error = []
    trials_error_sign = []
    trials_target_angle = []
    trials_target_distance = []

    for trial_idx, trial_behv in enumerate(trials_behv):
        trial_ts = trial_behv['continuous']['ts'][0][0].reshape(-1)
        t_mask = (trial_ts > 0) & (
            ~np.isnan(trial_behv['continuous']['ymp'][0][0].reshape(-1)))
        t_mask &= trial_ts < trial_behv['events']['t_stop'][0][0].reshape(-1)
        if t_mask.sum() > 0:
            # remove the first data point to avoid downsample error
            t_mask[np.where(t_mask == True)[0][0]] = False

        # task varaibles from data
        mx = trial_behv['continuous']['xmp'][0][0][t_mask]
        my = trial_behv['continuous']['ymp'][0][0][t_mask]
        fx = trial_behv['continuous']['xfp'][0][0][t_mask]
        fy = trial_behv['continuous']['yfp'][0][0][t_mask]
        eye_hor_theta = trial_behv['continuous']['yre'][0][0][t_mask] # use yle for left eye.
        eye_ver_theta = trial_behv['continuous']['zre'][0][0][t_mask]
        # print(len(eye_hor_theta), len(eye_ver_theta), len(mx))
        mv = trial_behv['continuous']['v'][0][0][t_mask].reshape(-1, 1)
        mw = trial_behv['continuous']['w'][0][0][t_mask].reshape(-1, 1)

        # some adjustment for screen distance
        sx = np.ones_like(fx)
        sy = np.ones_like(fy)
        if my.size > 0:
            fx = np.ones_like(fx) * fx[0]
            fy = np.ones_like(fy) * fy[0]
            sx *= mx[-1]
            sy *= my[-1]
            my = my + 30
            fy = fy + 30
            sy = sy + 30

        # some thing coudl be removed from there.
        dx = fx - mx; dy = fy - my
        rel_dist = np.sqrt(dx**2 + dy**2); rel_ang = np.rad2deg(np.arctan2(dy, dx))
        rel_dist_stop = np.sqrt((sx - mx)**2 + (sy - my)**2)
        abs_dist = np.sqrt(mx**2 + my**2); abs_ang = np.rad2deg(np.arctan2(my, mx))
        body_theta = np.deg2rad(np.cumsum(mw*-1) * DT + 90)
        body_x, body_y = mx.reshape(-1), my.reshape(-1)


        # skip bad trial
        if t_mask.sum() * DT > 3.5 or t_mask.sum() * DT < 0.6 or mv.max() < 50 or \
                abs_dist[-1] < np.sqrt(fx**2 + fy**2)[-1] * 0.3:
            continue

        # errors
        if my.size > 0:
            trials_error.append(rel_dist[-1][0])
            trials_error_sign.append(rel_dist[-1][0])
            trials_target_angle.append(
                np.rad2deg(np.arctan2(fy, fx))[-1][0] - 90)
            trials_target_distance.append(np.sqrt(fx**2 + fy**2)[-1][0])
            d1 = np.sqrt(fx**2 + fy**2)
            r1 = (fx**2 + fy**2) / (2*fx)
            radian1 = 2 * r1 * np.arcsin(d1 / (2 * r1))
            d2 = np.sqrt(mx**2 + my**2)
            r2 = (mx**2 + my**2) / (2*mx + 1e-8)
            radian2 = 2 * r2 * np.arcsin(d2 / (2 * r2 + 1e-8))
            sign = np.ones_like(rel_dist)
            sign[radian2 < radian1] = -1
            rel_dist = sign * rel_dist
            trials_error_sign[-1] = rel_dist[-1][0]
        else:
            trials_error.append(np.nan)
            trials_error_sign.append(np.nan)
            trials_target_angle.append(np.nan)
            trials_target_distance.append(np.nan)

        target_variable = np.hstack([rel_dist, rel_ang, abs_dist, abs_ang,
                                     eye_hor_theta, eye_ver_theta, 
                                     fx, fy, mx, my, mv, mw])

        target_variable_ds = downsample(target_variable, bin_size=bin_size)

        (rel_dist, rel_ang, abs_dist, abs_ang,
         eye_hor_theta, eye_ver_theta,
         fx, fy, mx, my, mv, mw) = zip(*target_variable_ds)
        body_theta = -np.deg2rad(np.cumsum(mw) * 0.1 - 90)
        body_x, body_y = np.array(mx).reshape(-1), np.array(my).reshape(-1)
        latent_ff_hori, latent_ff_vert = convert_location_to_angle(abs(np.array(rel_dist)).reshape(-1), np.array(fx).reshape(-1), np.array(fy).reshape(-1),
                                                                   np.array(body_theta), np.array(
                                                                       body_x), np.array(body_y),
                                                                   np.array(eye_hor_theta).reshape(-1), np.array(eye_ver_theta).reshape(-1), remove_pre=False)

        # df
        sessdata['session'].append(int(sess))
        sessdata['trial'].append(trial_idx)
        sessdata['fullon'].append(trial_behv['logical']['firefly_fullON'][0][0][0][0])
        sessdata['density'].append(
            (trial_behv['prs'][0][0]['floordensity'].item()))
        # sessdata['rel_dist'].append(rel_dist)
        # sessdata['rel_ang'].append(rel_ang)
        # sessdata['abs_dist'].append(abs_dist)
        # sessdata['abs_ang'].append(abs_ang)
        sessdata['eye_hori'].append(eye_hor_theta)
        sessdata['eye_vert'].append(eye_ver_theta)
        sessdata['ff_hori'].append(latent_ff_hori.reshape(-1))
        sessdata['ff_vert'].append(latent_ff_vert.reshape(-1))
        sessdata['fx'].append(fx)
        sessdata['fy'].append(fy)
        sessdata['mx'].append(mx)
        sessdata['my'].append(my)
        sessdata['mv'].append(mv)
        sessdata['mw'].append(mw)

        # neural
        activities = []  # activities for all neurons for 1 trial. shape: ts, neurons
        for trials_unit in trials_units:
            fire_ts = trials_unit['trials'][0][trial_idx][0].reshape(-1)
            if fire_ts.size > 0 and fire_ts[-1] >= trial_ts[-1]:
                fire_ts = fire_ts[:-1]
            activity = np.zeros_like(trial_ts)
            bin_indices = np.digitize(fire_ts, trial_ts)
            unique_bins, bin_counts = np.unique(
                bin_indices, return_counts=True)
            activity[unique_bins] = bin_counts
            activities.append(activity)

        activities = np.vstack(activities).T   # time * unit
        activities = activities[t_mask]
        activities = gaussian_filter1d(
            activities, sigma=4, axis=0)  # neural for each trial
        activityds = downsample(activities, bin_size=bin_size)

        for area in areas:
            area_mask = [v in area for v in units_area]
            if sum(area_mask) == 0:
                activity_ = np.nan
            else:
                activity_ = activityds[:, area_mask]  # area activity
            sessdata[area].append(activity_)

    sessdata['error'] += (trials_error)
    sessdata['error_sign'] += (trials_error_sign)
    sessdata['target_angle'] += (trials_target_angle)
    sessdata['target_distance'] += (trials_target_distance)

tmp = pd.DataFrame(sessdata)
df = pd.merge(df, tmp, on=['trial', 'session'], how='inner')

## IRC input data (state, action, task)

In [11]:
sessdata=defaultdict(list)

for sess in session:
    states, actions, tasks=[],[],[]
    sessdf=df[df.session==sess]
    trial_idces=sessdf.trial

    for trial_idx in trial_idces:
        trialdf=sessdf[sessdf.trial==trial_idx]
        trialdata=trialdf.iloc[0]
        # task
        taskx = (trialdata.fx[0] - trialdata.mx[0]).astype('float32'); tasky = (trialdata.fy[0] - trialdata.my[0]).astype('float32')
        tasks.append([tasky/worldscale,taskx/worldscale])
        # actions
        trialaction=np.stack([trialdata.mv,trialdata.mw]).T
        trialaction[:,0]=trialaction[:,0]/worldscale # v need reduce scale
        trialaction[:,1]=trialaction[:,1]/180*pi
        actions.append(trialaction.astype('float32'))

        # states from run the actions
        px, py, heading, v, w = 0,0,0,0,0
        log=[]
        for a in trialaction:
            px, py, heading, v, w=state_step2(px, py, heading, v, w, a, dt=0.1,userad=True)
            log.append([px, py, heading, v, w])
        px, py, heading, v, w=state_step2(px, py, heading, v, w, a, dt=0.1,userad=True)
        log.append([px, py, heading, v, w])
        trialstates=np.array(log)[1:]
        
        states.append(trialstates.astype('float32'))

        sessdata['session'].append(sess)
        sessdata['trial'].append(trial_idx)
        sessdata['state'].append(trialstates.astype('float32'))  
        sessdata['action'].append(trialaction.astype('float32'))  
        sessdata['task'].append([tasky/worldscale,taskx/worldscale])  

tmp=pd.DataFrame(sessdata)
df = pd.merge(df, tmp, on=['trial','session'], how='inner')

## Compute belief 

In [12]:
# model estimated likelihood (negative log likelihood)
torch.manual_seed(42)
arg = Config()

env = ffacc_real.FireFlyPaper(arg)
env.debug=True
phi = torch.tensor([[0.5],
                    [pi/2],
                    [0.001],
                    [0.001],
                    [0.001],
                    [0.001],
                    [0.13],
                    [0.001],
                    [0.001],
                    [0.001],
                    [0.001],
                    ])

agent_ = TD3.load(workdir/'trained_agent/paper')
agent = agent_.actor.mu.cpu()

In [13]:
thetas={}
for idensity in range(4):
    datapath = Path(resdir/f'{m}_mat_ruiyi/preirc_den_{idensity}')
    savename = datapath.parent/(f'{m}_{idensity}'+datapath.name)
    invfile=savename
    print(datapath.name)
    finaltheta, finalcov, err = process_inv(
        invfile, removegr=False, usingbest=False)
    print(finaltheta[:4])
    # finaltheta[0]=1
    # finaltheta[1]=1.3
    # finaltheta[1]=0.5
    # finaltheta[1]=0.2
    thetas[idensity]=finaltheta


preirc_den_0
/Users/yc/Documents/lab_data/m51_mat_ruiyi/m51_0preirc_den_0
using ind:  -1 final logll :  16.993248803274973
tensor([[0.9924],
        [0.7905],
        [0.7104],
        [0.1499]])
preirc_den_1
/Users/yc/Documents/lab_data/m51_mat_ruiyi/m51_1preirc_den_1
using ind:  -1 final logll :  15.665690626416888
tensor([[1.1921],
        [0.8381],
        [0.6041],
        [0.1469]])
preirc_den_2
/Users/yc/Documents/lab_data/m51_mat_ruiyi/m51_2preirc_den_2
using ind:  -1 final logll :  16.006959642682755
tensor([[1.1985],
        [0.9138],
        [0.5796],
        [0.1894]])
preirc_den_3
/Users/yc/Documents/lab_data/m51_mat_ruiyi/m51_3preirc_den_3
using ind:  -1 final logll :  15.557111876351494
tensor([[1.1422],
        [0.8885],
        [0.5519],
        [0.1930]])


## Compute likelihood
todo, need to do this trial by trial to take account of density.

In [14]:
# df likelihood
today='0615'
skipll=not(date.today().strftime("%m%d") == today)

def lltrial(state, action, task, finaltheta, samples=5):
    with torch.no_grad():
        return monkeyloss_(agent, action, np.array(task).reshape(1,-1), phi, finaltheta, env, action_var=0.01, num_iteration=1, states=state, samples=samples, gpu=False).item()

likelihood_df=defaultdict(list)

if not skipll: 
    # compute new ll and save with today date
    for sess in session:
        sessdf=df[df.session==sess]
        state, action, task = df.state.to_list(), df.action.to_list(),df.task.to_list()
        trial_idces=sessdf.trial
        for trial_idx in trial_idces:
            trialdf=sessdf[sessdf.trial==trial_idx]
            trialdata=trialdf.iloc[0]
            state, action, task = trialdata.state, trialdata.action, trialdata.task
            trial_likelihood=lltrial([state], [action], [task], finaltheta) 

            likelihood_df['session'].append(sess)
            likelihood_df['trial'].append(trial_idx)
            likelihood_df['likelihood'].append(trial_likelihood)
        
    notify('all done')
    today = date.today().strftime("%m%d") # mm/dd
    with open(resdir/f'{folder}/irc_ll_{today}','wb+') as f:
        pickle.dump(likelihood_df,f)
    notify('compute likelihood complete')
    print(f'computed likelihood saved to {folder}/irc_ll_{today}')

else: # use the given day
    print(f'use computed likelihood from day {today}')
    with open(resdir/f'{folder}/irc_ll_{today}','rb') as f:
            likelihood_df=pickle.load(f)

tmp=pd.DataFrame(likelihood_df)
df = pd.merge(df, tmp, on=['trial','session'], how='inner')

use computed likelihood from day 0615


In [15]:
denslookup={0.0001:0, 0.0005:1, 0.001:2,  0.005:3}
thisdensity=trialdf.density.item()
denslookup[thisdensity],thisdensity

(3, 0.005)

In [16]:
# df belief
today='0615'
skipblief=not(date.today().strftime("%m%d") == today)

def lltrial(state, action, task, finaltheta, samples=5):
    with torch.no_grad():
        return monkeyloss_(agent, action, np.array(task).reshape(1,-1), phi, finaltheta, env, action_var=0.01, num_iteration=1, states=state, samples=samples, gpu=False).item()

belief_df=defaultdict(list)

if not skipblief:
    for sess in session:
        sessdf=df[df.session==sess]
        state, action, task = df.state.to_list(), df.action.to_list(),df.task.to_list()
        trial_idces=sessdf.trial
        for trial_idx in trial_idces:
            trialdf=sessdf[sessdf.trial==trial_idx]
            trialdata=trialdf.iloc[0]
            state, action, task = trialdata.state, trialdata.action, trialdata.task
            thisdensity=trialdf.density.item()
            theta=thetas[denslookup[thisdensity]]

            _, _, ep_belief, ep_rawcov = run_trials(agent=agent, 
                                                   env=env, phi=phi, theta=theta,          task=task, ntrials=1,
                                                    pert=None, given_obs=None, return_belief=True, given_action=action, given_state=state)
            # trial info
            belief_df['session'].append(sess)
            belief_df['trial'].append(trial_idx)

            # belief
            if len(state)<5: # 
                belief_df['belief'].append(np.nan)
                belief_df['rawcov'].append(np.nan)
            else:
                init=torch.tensor(state[0]).reshape(-1,1)
                trial_belief=(ep_belief[0]-ep_belief[0][0]+init)
                belief_df['belief'].append(np.array(trial_belief)[:,:,0])
                belief_df['rawcov'].append(np.array(ep_rawcov[0]))
        
    notify('all done')
    today = date.today().strftime("%m%d") # mm/dd
    with open(resdir/f'{folder}/irc_belief_{today}','wb+') as f:
        pickle.dump(belief_df,f)
    notify('compute likelihood complete')
    print(f'computed likelihood saved to {folder}/irc_belief_{today}')

else: # use the given day
    print(f'use computed likelihood from day {today}')
    with open(resdir/f'{folder}/irc_belief_{today}','rb') as f:
            belief_df=pickle.load(f)
tmp=pd.DataFrame(belief_df)
tmp=tmp.rename(columns={'cov': 'rawcov'})
df = pd.merge(df, tmp, on=['trial','session'], how='inner')

use computed likelihood from day 0615


In [17]:
df.columns, tmp.columns

(Index(['trial', 'session', 'fullon', 'density', 'eye_hori', 'eye_vert',
        'ff_hori', 'ff_vert', 'fx', 'fy', 'mx', 'my', 'mv', 'mw', 'PPC', 'PFC',
        'MST', 'error', 'error_sign', 'target_angle', 'target_distance',
        'state', 'action', 'task', 'likelihood', 'belief', 'rawcov'],
       dtype='object'),
 Index(['session', 'trial', 'belief', 'rawcov'], dtype='object'))

In [18]:
# unpack the belief state
df['bmx']=df.apply(lambda x:x.belief[:,1]*worldscale, axis=1)
df['bmy']=df.apply(lambda x:x.belief[:,0]*worldscale, axis=1)
df['heading']=df.apply(lambda x: x.state[:,2]*180/pi, axis=1)
df['belief_heading']=df.apply(lambda x: x.belief[:,2]*180/pi, axis=1)
df['timer']=df.apply(lambda x: np.arange(len(x.mx)), axis=1)
df['countdown']=df.apply(lambda x: np.flip(-np.arange(len(x.mx)), axis=0), axis=1)

In [19]:
sessdata=defaultdict(list)

for sess in session:
    sessdf=df[df.session==sess]
    states, actions, tasks = sessdf.state.to_list(), sessdf.action.to_list(),sessdf.task.to_list()
    beliefs,rawcovs=sessdf['belief'].to_list(),sessdf['rawcov'].to_list()

    sess_latentff_hori, sess_latentff_vert = [], []
    for ep_beliefs, ep_rawcovs, task in zip(beliefs, rawcovs, tasks): # process for each trial
        mx, my, body_theta,  mv, mw = zip(*ep_beliefs)
        body_theta = -(np.cumsum(mw) * 0.1-pi/2)
        body_x, body_y = np.asarray(my).reshape(-1).astype('float') * \
            worldscale, np.asarray(mx).reshape(-1).astype('float')*worldscale

        fx, fy = task[1]*worldscale, task[0]*worldscale
        rel_dist = ((fx-body_x)**2+(fy-body_y)**2)**0.5
        hor_theta_, ver_theta_ = convert_location_to_angle(abs(np.array(rel_dist)).reshape(-1).astype('float'), np.array(fx).reshape(-1).astype('float'), np.array(fy).reshape(-1).astype('float'),
                                                           body_theta.astype('float'), body_x.astype(
                                                               'float'), body_y.astype('float'),
                                                           np.array(rel_dist).reshape(-1).astype('float'), # use the true eye positions to remove pre saccade movement and after overshooting eye movement
                                                           np.array(rel_dist).reshape(-1).astype('float'), DT=0.1, remove_pre=False, remove_post=False)
        # plt.plot(hor_theta_, ver_theta_, 'g')
        sess_latentff_hori.append(hor_theta_)
        sess_latentff_vert.append(ver_theta_)
    sessdata['belief_ff_hori']+=[a.reshape(-1) for a in sess_latentff_hori]
    sessdata['belief_ff_vert']+=[a.reshape(-1) for a in sess_latentff_vert]
    

# TODO temp need change to merge
df['belief_ff_hori']=sessdata['belief_ff_hori']
df['belief_ff_vert']=sessdata['belief_ff_vert']

In [20]:
# angle from start
def get_angle_from_start(row):
    return np.arctan2((np.array(row.my)), (np.array(row.mx)))

df['angle_from_start']=df.apply(get_angle_from_start, axis=1)

def get_belief_angle_from_start(row):
    return np.arctan2((np.array(row.bmy)), (np.array(row.bmx)))

df['belief_angle_from_start']=df.apply(get_angle_from_start, axis=1)


In [21]:
# check len
random_rows = df.sample(n=3)

# Iterate over the columns
for col in random_rows.columns:
    try:
        # Get the column values for the selected rows
        col_values = random_rows[col]
        # Calculate the length of each column value
        lengths = col_values.apply(lambda x: len(x))
        # Print column name and the length of each column value
        print(f"Column: {col}, Lengths: {lengths.tolist()}")
    except: continue


Column: eye_hori, Lengths: [25, 22, 15]
Column: eye_vert, Lengths: [25, 22, 15]
Column: ff_hori, Lengths: [25, 22, 15]
Column: ff_vert, Lengths: [25, 22, 15]
Column: fx, Lengths: [25, 22, 15]
Column: fy, Lengths: [25, 22, 15]
Column: mx, Lengths: [25, 22, 15]
Column: my, Lengths: [25, 22, 15]
Column: mv, Lengths: [25, 22, 15]
Column: mw, Lengths: [25, 22, 15]
Column: PPC, Lengths: [25, 22, 15]
Column: state, Lengths: [25, 22, 15]
Column: action, Lengths: [25, 22, 15]
Column: task, Lengths: [2, 2, 2]
Column: belief, Lengths: [25, 22, 15]
Column: rawcov, Lengths: [25, 22, 15]
Column: bmx, Lengths: [25, 22, 15]
Column: bmy, Lengths: [25, 22, 15]
Column: heading, Lengths: [25, 22, 15]
Column: belief_heading, Lengths: [25, 22, 15]
Column: timer, Lengths: [25, 22, 15]
Column: countdown, Lengths: [25, 22, 15]
Column: belief_ff_hori, Lengths: [25, 22, 15]
Column: belief_ff_vert, Lengths: [25, 22, 15]
Column: angle_from_start, Lengths: [25, 22, 15]
Column: belief_angle_from_start, Lengths: [25,

In [22]:
# check size
random_rows = df.sample(n=3)

# Iterate over the columns
for col in random_rows.columns:
    try:
        # Get the column values for the selected rows
        col_values = random_rows[col]
        # Calculate the length of each column value
        lengths = col_values.apply(lambda x: (x.shape))
        # Print column name and the length of each column value
        print(f"Column: {col}, Lengths: {lengths.tolist()}")
    except: continue

Column: ff_hori, Lengths: [(16,), (22,), (10,)]
Column: ff_vert, Lengths: [(16,), (22,), (10,)]
Column: PPC, Lengths: [(16, 94), (22, 112), (10, 94)]
Column: state, Lengths: [(16, 5), (22, 5), (10, 5)]
Column: action, Lengths: [(16, 2), (22, 2), (10, 2)]
Column: belief, Lengths: [(16, 5), (22, 5), (10, 5)]
Column: rawcov, Lengths: [(16, 5, 5), (22, 5, 5), (10, 5, 5)]
Column: bmx, Lengths: [(16,), (22,), (10,)]
Column: bmy, Lengths: [(16,), (22,), (10,)]
Column: heading, Lengths: [(16,), (22,), (10,)]
Column: belief_heading, Lengths: [(16,), (22,), (10,)]
Column: timer, Lengths: [(16,), (22,), (10,)]
Column: countdown, Lengths: [(16,), (22,), (10,)]
Column: belief_ff_hori, Lengths: [(16,), (22,), (10,)]
Column: belief_ff_vert, Lengths: [(16,), (22,), (10,)]
Column: angle_from_start, Lengths: [(16,), (22,), (10,)]
Column: belief_angle_from_start, Lengths: [(16,), (22,), (10,)]


## the varialbes we have:

In [23]:
# state, change coord example
trialdf=df.iloc[10]
mx, my,mw, fx,fy=trialdf.mx, trialdf.my, trialdf.mw, trialdf.fx, trialdf.fy
mx, my,mw, fx,fy=[np.array(a) for a in [mx, my,mw, fx,fy]]
sx = np.ones_like(fx)
sy = np.ones_like(fy)
if my.size > 0:
    fx = np.ones_like(fx) * fx[0]
    fy = np.ones_like(fy) * fy[0]
    sx *= mx[-1]
    sy *= my[-1]
    my = my + 30
    fy = fy + 30
    sy = sy + 30
dx = fx - mx; dy = fy - my
rel_dist = np.sqrt(dx**2 + dy**2); rel_ang = np.rad2deg(np.arctan2(dy, dx))
# rel_dist_stop = np.sqrt((sx - mx)**2 + (sy - my)**2)
# abs_dist = np.sqrt(mx**2 + my**2); abs_ang = np.rad2deg(np.arctan2(my, mx))
heading = np.deg2rad(np.cumsum(mw*-1) * 0.1 + 90) 

latent_ff_hori, latent_ff_vert = convert_location_to_angle(
    rel_dist,
    fx,
    fy,
    heading,
    mx,
    my,
    None, # not needed if remove pre = false
    None,
    remove_pre=False
)
# plt.scatter(latent_ff_hori,trialdf.ff_hori); plt.title('state');plt.axis('equal'); plt.show()


In [24]:
# # belief chnage coord example
trialdf=df.iloc[23]

bmx, bmy, bmw, fx,fy = trialdf.bmx, trialdf.bmy, np.array(trialdf.mw), trialdf.fx, trialdf.fy
ep_beliefs=trialdf.belief

mx, my, body_theta,  mv, mw = zip(*ep_beliefs)
mx, my,mw, fx,fy=[np.array(a) for a in [mx, my,mw, fx,fy]]
mx,my=my*worldscale, mx*worldscale

# plt.scatter(mx, bmx); plt.title('beliefmx vs bmx');plt.axis('equal'); plt.show()

sx = np.ones_like(fx)
sy = np.ones_like(fy)
if my.size > 0:
    fx = np.ones_like(fx) * fx[0]
    fy = np.ones_like(fy) * fy[0]
    sx *= mx[-1]
    sy *= my[-1]
    my = my + 30
    fy = fy + 30
    sy = sy + 30
dx = fx - mx; dy = fy - my
rel_dist = np.sqrt(dx**2 + dy**2); rel_ang = np.rad2deg(np.arctan2(dy, dx))
# rel_dist_stop = np.sqrt((sx - mx)**2 + (sy - my)**2)
# abs_dist = np.sqrt(mx**2 + my**2); abs_ang = np.rad2deg(np.arctan2(my, mx))

heading2=-(np.cumsum(mw) * 0.1-pi/2)

latent_ff_hori, latent_ff_vert = convert_location_to_angle(
    rel_dist,
    fx,
    fy,
    heading2,
    mx,
    my,
    None, # not needed if remove pre = false
    None,
    remove_pre=False
)
# plt.scatter(latent_ff_hori,trialdf.belief_ff_hori); plt.title('belief');plt.axis('equal'); plt.show()


In [25]:
# convert belief rawcov example
trialdf=df.iloc[23]

bmx, bmy, bmw, fx,fy = trialdf.bmx, trialdf.bmy, np.array(trialdf.mw), trialdf.fx, trialdf.fy
rawcov=trialdf['rawcov'][:,:2,:2]


belief_heading=trialdf.belief_heading
rotdegree=belief_heading+180
relativeposrawcov=[]
for degree, thisrawcov in zip(rotdegree, rawcov):
    R=np.array([[np.cos(-degree/180*pi),-np.sin(-degree/180*pi)],[np.sin(-degree/180*pi),np.cos(-degree/180*pi)]])
    relativeposrawcov.append(R.T@thisrawcov[:2,:2]@R)
relativeposrawcov=np.stack(relativeposrawcov)
relativeposrawcov.shape
rotdegree

array([180.00067, 180.00067, 179.62195, 179.36865, 179.58482, 181.6971 ,
       186.22366, 191.78711, 196.95834, 201.84085, 204.80537, 205.76024,
       206.60269, 207.36252, 208.06198, 209.04297, 209.75887, 211.42252],
      dtype=float32)

In [69]:
# rotate belief cov df
def fun(trialdf):
    if trialdf.fullon==1: # always on target, assign zero uncertainty
        return np.ones_like(trialdf['rawcov'][:,:2,:2])*1e-6
    cov=trialdf['rawcov'][:,:2,:2]
    belief_heading=trialdf.belief_heading
    rotdegree=belief_heading+180
    relativeposcov=[]
    for degree, thiscov in zip(rotdegree, cov):
        R=np.array([[np.cos(-degree/180*pi),-np.sin(-degree/180*pi)],[np.sin(-degree/180*pi),np.cos(-degree/180*pi)]])
        relativeposcov.append(R.T@thiscov[:2,:2]@R)
    relativeposcov=np.stack(relativeposcov)*worldscale*worldscale
    return relativeposcov
df['relcov']=df.apply(fun, axis=1)

In [34]:
date='0918'
df.to_pickle(resdir/f'{date}_m51df.pkl')
notify(f'saved: {date}_m51df.pkl')
print(f'saved: {date}_m51df.pkl')

saved: 0918_m51df.pkl
